# SageMaker Example

## 2. Build the container

demo codes are in `app/`
build and push the docker with following commands:

In [ ]:
!bash build_and_push_sglang.sh

## 3. Deploy on SageMaker

define the model and deploy on SageMaker


### 3.1 Init SageMaker session

In [ ]:
# !pip install boto3 sagemaker transformers
import re
import json
import os,dotenv
import boto3
import sagemaker
from sagemaker import Model


dotenv.load_dotenv()
print(os.environ)

boto_sess = boto3.Session(
    region_name='us-east-1'
)

sess = sagemaker.session.Session(boto_session=boto_sess)
# role = sagemaker.get_execution_role()
role = os.environ.get('role')

### 3.2 Prepare model file

#### Option 2: deploy vllm by model_id

In [ ]:
!tar czvf model.tar.gz model_tar/

In [ ]:


s3_code_prefix = f"sagemaker_endpoint/sglang/"
bucket = sess.default_bucket() 
code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

### 3.3 Deploy model

### vllm

In [ ]:
CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/vllm:v0.7.2'

env={
    "HF_MODEL_ID": model_name,
    "DTYPE": dtype,
    "LIMIT_MM_PER_PROMPT":extra_params.get('limit_mm_per_prompt',''),
    "S3_MODEL_PATH":model_path,
    "VLLM_ALLOW_LONG_MAX_MODEL_LEN":"1",
    "HF_TOKEN":os.environ.get('HUGGING_FACE_HUB_TOKEN'),
    "MAX_MODEL_LEN":extra_params.get('max_model_len', "12288"), 
    "ENABLE_PREFIX_CACHING": "1" if extra_params.get('enable_prefix_caching') else "0",
    "TENSOR_PARALLEL_SIZE": extra_params.get('tensor_parallel_size',str(get_auto_tensor_parallel_size(instance_type))),
    "MAX_NUM_SEQS": extra_params.get('max_num_seqs','256'),
    "ENFORCE_EAGER": "1" if extra_params.get('enforce_eager') else "0",

        }

model = Model(
    name=sagemaker.utils.name_from_base("sagemaker-vllm")+"_model",
    model_data=code_artifact,
    image_uri=CONTAINER,
    role=role,
    sagemaker_session=sess,
    env=env,
    
)



# 部署模型到endpoint
endpoint_name = sagemaker.utils.name_from_base("sagemaker-vllm")+"_endpoint"
print(f"endpoint_name: {endpoint_name}")
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
)

### SGLANG

In [ ]:
from sagemaker.enums import EndpointType
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements
from sagemaker import Predictor
from sagemaker import Model


CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/sglang:v0.4.3.post2-cu124'
model_path = "s3://sagemaker-us-east-1-434444145045/Qwen2-5-3B-Instruct/032650faedac452e86f95f3f3b004342/finetuned_model/"
# model_id = 'Qwen/Qwen2-1.5B-Instruct'
env={
    # "HF_MODEL_ID": model_id,
    "S3_MODEL_PATH":model_path,
}

model_name = sagemaker.utils.name_from_base("sagemaker-sglang")+"-model"

model = Model(
    name=model_name,
    model_data=code_artifact,
    image_uri=CONTAINER,
    role=role,
    sagemaker_session=sess,
    env=env,
    predictor_cls = Predictor,
)


# 部署模型到endpoint
endpoint_name = sagemaker.utils.name_from_base("sagemaker-sglang")+"-endpoint"
print(f"endpoint_name: {endpoint_name}")
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    model_name=model_name, 
)

## 4. Test

you can invoke your model with SageMaker SDK

### 4.1 Message api non-stream mode

In [ ]:
runtime = boto3.client('runtime.sagemaker',region_name='us-east-1')
payload = {
    "messages": [
    {
        "role": "user",
        "content": "who are you"
    }
    ],
    "model":"qwen",
    "max_tokens": 1024,
    "stream": False
}
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])

### 4.2 Message api stream mode

In [ ]:
payload = {
    "messages": [
    {
        "role": "user",
        "content": "Write a quick sort in python"
    }
    ],
    "model":"custome",
    "max_tokens": 4096,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.span()[1]
            print(data["choices"][0]["delta"]["content"], end="")
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]